# Polity5

In [38]:
import pandas as pd
polity_df = pd.read_excel('../datasets/raw/democracy/polity/POLITY5-PRC.xlsx', sheet_name='Data')

# Rename columns
polity_df.rename(columns={'Economy ISO3': 'country_iso', 'Economy Name': 'country_name'}, inplace=True)

polity_df['Indicator ID'] = polity_df['Indicator ID'].str.replace(r'^POLITY5\.PRC\.', '', regex=True)


# Drop the columns Attribute 1	Attribute 2	Attribute 3
polity_df.drop(columns=['Attribute 1', 'Attribute 2', 'Attribute 3'], inplace=True)

# Set the years as identifiers instead of columns
years = [str(year) for year in range(1776, 2019)]
polity_df_long = polity_df.melt(
    id_vars=['country_iso', 'country_name', 'Indicator ID'],
    value_vars=years,
    var_name='Year',
    value_name='Value'
)

polity_df_long['Year'] = pd.to_numeric(polity_df_long['Year'])

# Set the Indicator as the column
polity_df_final = polity_df_long.pivot_table(
    index=['country_iso', 'country_name', 'Year'],
    columns='Indicator ID',
    values='Value'
).reset_index()

polity_df_final.to_csv('../datasets/processed/democracy/polity/polity5.csv', index=False)

polity_df_final.head()

Indicator ID,country_iso,country_name,Year,autoc,democ,durable,exconst,exrec,parcomp,parreg,polcomp,polity,polity2,xconst,xrcomp,xropen,xrreg
0,AFG,Afghanistan,1800,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0
1,AFG,Afghanistan,1801,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0
2,AFG,Afghanistan,1802,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0
3,AFG,Afghanistan,1803,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0
4,AFG,Afghanistan,1804,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0


In [39]:
countries = polity_df_final["country_name"].unique()

print(f"Nr of countries: {len(countries)} - Countries: {', '.join(countries)}")


Nr of countries: 166 - Countries: Afghanistan, Angola, Albania, United Arab Emirates, Argentina, Armenia, Australia, Austria, Azerbaijan, Burundi, Belgium, Benin, Burkina Faso, Bangladesh, Bulgaria, Bahrain, Bosnia and Herzegovina, Belarus, Bolivia, Brazil, Bhutan, Botswana, Central African Republic, Canada, Switzerland, Chile, China, Cote d'Ivoire, Cameroon, Congo, Rep., Colombia, Comoros, Cabo Verde, Costa Rica, Cuba, Cyprus, Czechia, Germany, Djibouti, Denmark, Dominican Republic, Algeria, Ecuador, Egypt, Arab Rep., Eritrea, Spain, Estonia, Ethiopia, Finland, Fiji, France, Gabon, United Kingdom, Georgia, Ghana, Guinea, Gambia, The, Guinea-Bissau, Equatorial Guinea, Greece, Guatemala, Guyana, Honduras, Croatia, Haiti, Hungary, Indonesia, India, Ireland, Iran, Islamic Rep., Iraq, Israel, Italy, Jamaica, Jordan, Japan, Kazakhstan, Kenya, Kyrgyz Republic, Cambodia, Korea, Rep., Kuwait, Lao PDR, Lebanon, Liberia, Libya, Sri Lanka, Lesotho, Lithuania, Luxembourg, Latvia, Morocco, Moldova,

# Freedom in the World

In [40]:
freedom_df = pd.read_excel('../datasets/raw/democracy/freedom-world/All_data_FIW_2013-2024.xlsx', sheet_name='FIW13-25', header=1)

freedom_df.rename(columns={'Country/Territory': 'country', 'Edition': 'year'}, inplace=True)
freedom_df.drop(columns=['Region'], inplace=True)
freedom_df = freedom_df[freedom_df['C/T'] == 'c']
freedom_df.drop(columns=['Add Q', 'Add A', 'C/T'], inplace=True)
freedom_df.columns = freedom_df.columns.str.strip().str.lower().str.replace(' ', '_')
freedom_df.to_csv('../datasets/processed/democracy/freedom-world/freedom-world.csv', index=False)
freedom_df.head()

,country,year,status,pr_rating,cl_rating,a1,a2,a3,a,b1,...,f3,f4,f,g1,g2,g3,g4,g,cl,total
1,Afghanistan,2025,NF,7,7,0,0,0,0,0,...,0,0,0,0,1,0,1,2,5,6
2,Albania,2025,PF,3,3,3,3,3,9,3,...,2,3,10,3,2,2,2,9,40,68
3,Algeria,2025,NF,6,5,1,1,1,3,1,...,2,2,6,2,2,2,1,7,21,31
4,Andorra,2025,F,1,1,4,4,4,12,4,...,4,3,15,4,4,3,4,15,55,93
5,Angola,2025,NF,6,5,0,2,1,3,1,...,1,2,5,1,1,1,0,3,18,28


In [41]:
countries = freedom_df["country"].unique()

print(f"Nr of countries: {len(countries)} - Countries: {', '.join(countries)}")

print(f"From years {freedom_df['year'].min()} to {freedom_df['year'].max()}")


Nr of countries: 195 - Countries: Afghanistan, Albania, Algeria, Andorra, Angola, Antigua and Barbuda, Argentina, Armenia, Australia, Austria, Azerbaijan, Bahamas, Bahrain, Bangladesh, Barbados, Belarus, Belgium, Belize, Benin, Bhutan, Bolivia, Bosnia and Herzegovina, Botswana, Brazil, Brunei, Bulgaria, Burkina Faso, Burundi, Cabo Verde, Cambodia, Cameroon, Canada, Central African Republic, Chad, Chile, China, Colombia, Comoros, Congo (Brazzaville), Congo (Kinshasa), Costa Rica, Cote d'Ivoire, Croatia, Cuba, Cyprus, Czech Republic, Denmark, Djibouti, Dominica, Dominican Republic, Ecuador, Egypt, El Salvador, Equatorial Guinea, Eritrea, Estonia, Eswatini, Ethiopia, Fiji, Finland, France, Gabon, Georgia, Germany, Ghana, Greece, Grenada, Guatemala, Guinea, Guinea-Bissau, Guyana, Haiti, Honduras, Hungary, Iceland, India, Indonesia, Iran, Iraq, Ireland, Israel, Italy, Jamaica, Japan, Jordan, Kazakhstan, Kenya, Kiribati, Kosovo, Kuwait, Kyrgyzstan, Laos, Latvia, Lebanon, Lesotho, Liberia, Li

# Autocratic Regime Data

In [42]:
autocratic_cases_df = pd.read_excel('../datasets/raw/democracy/autocratic/GWF Autocratic Regimes 1.2/GWF Autocratic Regimes.xlsx', sheet_name='Autocratic Regimes Case List', header=0)

# Transform the dates to datetime in day/month/year format. By default Pandas uses month/day/year format.
autocratic_cases_df['gwf_startdate'] = pd.to_datetime(autocratic_cases_df['gwf_startdate'], dayfirst=True, errors='coerce')
autocratic_cases_df['gwf_enddate'] = pd.to_datetime(autocratic_cases_df['gwf_enddate'], dayfirst=True, errors='coerce')

# gwf_endyr is parsed as a float by default, so we need to convert it to an integer
# If values are not integers, set them to -1 (For example, for cases where the autocratic regime is still ongoing)
autocratic_cases_df['gwf_startyr'] = autocratic_cases_df['gwf_startyr'].fillna(-1).astype(int)
autocratic_cases_df['gwf_endyr'] = autocratic_cases_df['gwf_endyr'].fillna(-1).astype(int)

autocratic_cases_df.to_csv('../datasets/processed/democracy/autocratic/autocratic-regime-case-list.csv', index=False)
autocratic_cases_df.head()

,cowcode,gwf_country,gwf_casename,gwf_startdate,gwf_enddate,gwf_startyr,gwf_endyr,gwf_subsreg,gwf_howend,gwf_violent,gwf_regimetype
0,700,Afghanistan,Afghanistan 29-73,1929-10-10,1973-07-17,1930,1973,2,5,2,monarchy
1,700,Afghanistan,Afghanistan 73-78,1973-07-17,1978-04-27,1974,1978,2,5,4,personal
2,700,Afghanistan,Afghanistan 78-92,1978-04-27,1992-04-16,1979,1992,3,6,4,party-personal
3,700,Afghanistan,Afghanistan 96-01,1996-09-27,2001-11-13,1997,2001,3,7,4,party-based
4,700,Afghanistan,Afghanistan 09-NA,2009-08-20,2010-12-31,2010,-1,0,0,0,personal


In [43]:
# Now we want to process the TSCS Data, which contains the autocratic regimes data over the years
autocratic_years_df = pd.read_excel('../datasets/raw/democracy/autocratic/GWF Autocratic Regimes 1.2/GWF Autocratic Regimes.xlsx', sheet_name='TSCS data', header=0)

# Transform the dates to datetime in day/month/year format. By default Pandas uses month/day/year format.
autocratic_years_df['gwf_startdate'] = pd.to_datetime(autocratic_years_df['gwf_startdate'], dayfirst=True, errors='coerce')
autocratic_years_df['gwf_enddate'] = pd.to_datetime(autocratic_years_df['gwf_enddate'], dayfirst=True, errors='coerce')

autocratic_years_df.to_csv('../datasets/processed/democracy/autocratic/autocratic-regime-country-years.csv', index=False)
autocratic_years_df.head()

,cowcode,year,gwf_country,gwf_casename,gwf_startdate,gwf_enddate,gwf_spell,gwf_duration,gwf_fail,gwf_fail_subsregime,gwf_fail_type,gwf_fail_violent,gwf_regimetype,gwf_party,gwf_personal,gwf_military,gwf_monarch
0,40,1953,Cuba,Cuba 52-59,1952-10-03,1959-01-01,7,1,0,0,0,0,personal,0,1,0,0
1,40,1954,Cuba,Cuba 52-59,1952-10-03,1959-01-01,7,2,0,0,0,0,personal,0,1,0,0
2,40,1955,Cuba,Cuba 52-59,1952-10-03,1959-01-01,7,3,0,0,0,0,personal,0,1,0,0
3,40,1956,Cuba,Cuba 52-59,1952-10-03,1959-01-01,7,4,0,0,0,0,personal,0,1,0,0
4,40,1957,Cuba,Cuba 52-59,1952-10-03,1959-01-01,7,5,0,0,0,0,personal,0,1,0,0


In [44]:
countries = autocratic_cases_df["gwf_country"].unique()

print(f"Autocratic Regimes - Nr of countries: {len(countries)} - Countries: {', '.join(countries)}")
print(f"Autocratic Regimes - From years {autocratic_cases_df['gwf_startyr'].min()} to {autocratic_cases_df['gwf_endyr'].max()}")

countries = autocratic_years_df["gwf_country"].unique()

print(f"Autocratic Years - Nr of countries: {len(countries)} - Countries: {', '.join(countries)}")
print(f"Autocratic Years - From years {autocratic_years_df['year'].min()} to {autocratic_years_df['year'].max()}")


Autocratic Regimes - Nr of countries: 120 - Countries: Afghanistan, Albania, Algeria, Angola, Argentina, Armenia, Azerbaijan, Bangladesh, Belarus, Benin, Bolivia, Botswana, Brazil, Bulgaria, Burkina Faso, Burundi, Cambodia, Cameroon, Cen African Rep, Chad, Chile, China, Colombia, Congo-Brz, Congo/Zaire, Costa Rica, Cuba, Czechoslovakia, Dominican Rep, Ecuador, Egypt, El Salvador, Eritrea, Ethiopia, Gabon, Gambia, Georgia, Germany East, Ghana, Greece, Guatemala, Guinea, Guinea Bissau, Haiti, Honduras, Hungary, Indonesia, Iran, Iraq, Ivory Coast, Jordan, Kazakhstan, Kenya, Korea North, Korea South, Kuwait, Kyrgyzstan, Laos, Lesotho, Liberia, Libya, Madagascar, Malawi, Malaysia, Mali, Mauritania, Mexico, Mongolia, Morocco, Mozambique, Myanmar, Namibia, Nepal, Nicaragua, Niger, Nigeria, Oman, Pakistan, Panama, Paraguay, Peru, Philippines, Poland, Portugal, Romania, Russia, Rwanda, Saudi Arabia, Senegal, Serbia, Sierra Leone, Singapore, Somalia, South Africa, Soviet Union, Spain, Sri Lanka,

# Boix-Miller-Rosato Dichotomous Coding of Democracy

In [78]:
# Load CSV
dichotomous_df = pd.read_csv('../datasets/raw/democracy/dichotomous/democracy-v4.0.csv')

# If democracy_omitteddata is NA, set it to -1
dichotomous_df['democracy_omitteddata'] = dichotomous_df['democracy_omitteddata'].fillna(-1)

# Some country aren't modern countries (E.g.: Bavaria), if the abbreviation field is not 3 characters, drop the row
dichotomous_df = dichotomous_df[dichotomous_df['abbreviation'].str.len() == 3]

# Some countries don't have democracy data (They should), filter those out
# Set NA values to -1
dichotomous_df['democracy'] = dichotomous_df['democracy'].fillna(-1).astype(int)
# Filter out rows where democracy is not an integer
dichotomous_df = dichotomous_df[dichotomous_df['democracy'] != -1]
# Do the same for the other fields, if they have NA values, set them to -1 and filter them
dichotomous_df['democracy_trans'] = dichotomous_df['democracy_trans'].astype(int)
dichotomous_df['democracy_breakdowns'] = dichotomous_df['democracy_breakdowns'].astype(int)
dichotomous_df['democracy_duration'] = dichotomous_df['democracy_duration'].astype(int)
dichotomous_df['democracy_femalesuffrage'] = dichotomous_df['democracy_femalesuffrage'].astype(int)
# Column democracy_omitteddata is allowed to have NA values, set them to -1 and leave them as is
dichotomous_df['democracy_omitteddata'] = dichotomous_df['democracy_omitteddata'].fillna(-1).astype(int)

dichotomous_df.to_csv('../datasets/processed/democracy/dichotomous/dichotomous-democracy.csv', index=False)
dichotomous_df.head()

,country,ccode,abbreviation,abbreviation_undp,year,democracy,democracy_trans,democracy_breakdowns,democracy_duration,democracy_omitteddata,democracy_femalesuffrage
0,UNITED STATES OF AMERICA,2,USA,USA,1800,1,0,0,1,1,0
1,UNITED STATES OF AMERICA,2,USA,USA,1801,1,0,0,2,1,0
2,UNITED STATES OF AMERICA,2,USA,USA,1802,1,0,0,3,1,0
3,UNITED STATES OF AMERICA,2,USA,USA,1803,1,0,0,4,1,0
4,UNITED STATES OF AMERICA,2,USA,USA,1804,1,0,0,5,1,0


In [79]:
countries = dichotomous_df["country"].unique()

print(f"Dichotomous Democracy - Nr of countries: {len(countries)} - Countries: {', '.join(countries)}")
print(f"Dichotomous Democracy - From years {dichotomous_df['year'].min()} to {dichotomous_df['year'].max()}")


Dichotomous Democracy - Nr of countries: 203 - Countries: UNITED STATES OF AMERICA, CANADA, BAHAMAS, CUBA, HAITI, DOMINICAN REPUBLIC, JAMAICA, TRINIDAD & TOBAGO, BARBADOS, DOMINICA, GRENADA, ST. LUCIA, ST. VINCENT & GREN., ANTIGUA, ST. KITTS & NEVIS, MEXICO, BELIZE, GUATEMALA, HONDURAS, EL SALVADOR, NICARAGUA, COSTA RICA, PANAMA, COLOMBIA, VENEZUELA, GUYANA, SURINAME, ECUADOR, PERU, BRAZIL, BOLIVIA, PARAGUAY, CHILE, ARGENTINA, URUGUAY, UNITED KINGDOM, IRELAND, NETHERLANDS, BELGIUM, LUXEMBOURG, FRANCE, MONACO, LIECHSTENSTEIN, SWITZERLAND, SPAIN, ANDORRA, PORTUGAL, GERMANY, GERMANY, WEST, GERMANY, EAST, POLAND, AUSTRIA, HUNGARY, CZECHOSLOVAKIA, CZECH REPUBLIC, SLOVAKIA, ITALY, SAN MARINO, MALTA, ALBANIA, MACEDONIA, CROATIA, YUGOSLAVIA, BOSNIA, YUGOSLAVIA, FED. REP., SLOVENIA, GREECE, CYPRUS, BULGARIA, MOLDOVA, ROMANIA, USSR, RUSSIA, ESTONIA, LATVIA, LITHUANIA, UKRAINE, BELARUS, ARMENIA, GEORGIA, AZERBAIJAN, FINLAND, SWEDEN, NORWAY, DENMARK, ICELAND, CAPE VERDE, SAO TOME & PRINCIPE, GUINE

# Lexical Index of Electoral Democracy (LIED)

In [90]:
# Load CSV
lied_df = pd.read_excel('../datasets/raw/democracy/lied/LIED_6.6.xlsx', sheet_name='Sheet1', header=0)

# Drop the regions column as it's not part of the codebook and it's included in the version 6.6 only
lied_df.drop(columns=['region'], inplace=True)

# Transform vdem to int (If values are not integers, set them to -1)
lied_df['vdem'] = lied_df['vdem'].fillna(-1).astype(int)

# Drop the rows where vdem is -1
# The countries without vdem are not really countries and will be out of our analysis
lied_df = lied_df[lied_df['vdem'] != -1]

# Transform the year to int
# There is one record with an invalid quote in the year -> ¨2023, we transform it to 2023
lied_df['year'] = lied_df['year'].str.replace('¨', '').astype(int)

lied_df[lied_df['year'] == -1]

# lied_df.info()

ValueError: invalid literal for int() with base 10: '¨2023'